# 01.상품구매 데이터

## 01)데이터 불러오기

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

%matplotlib inline

In [4]:
buy = pd.read_csv('/content/buy.csv')
buy.columns = ['age','paycheck','purchase'] # age, paycheck : 연속, purchase :  범주 
buy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   age       20 non-null     int64 
 1   paycheck  20 non-null     int64 
 2   purchase  20 non-null     object
dtypes: int64(2), object(1)
memory usage: 608.0+ bytes


In [5]:
buy.head()

,age,paycheck,purchase
0,26,160,구매
1,35,210,비구매
2,26,220,비구매
3,29,260,구매
4,22,110,비구매


In [6]:
buy.purchase.value_counts()

비구매     11
구매       8
비구매      1
Name: purchase, dtype: int64

In [13]:
buy.purchase=buy.purchase.replace('비구매 ', '비구매')
buy.purchase.value_counts()

비구매    12
구매      8
Name: purchase, dtype: int64

## 02)KNN classification

In [15]:
# 피쳐 및 라벨 지정
X = buy.iloc[:,[0,1]]
y = buy.iloc[:,2]

In [16]:
# 피쳐 value 정규화
scaler = StandardScaler()
X = scaler.fit(X).transform(X)

In [17]:
# 분류모델 학습 및 예측
classifier = KNeighborsClassifier(n_neighbors = 2)
classifier.fit(X, y)

KNeighborsClassifier(n_neighbors=2)

In [18]:
# 실제 피쳐값으로 예측 라벨값 생성 
y_pred = classifier.predict(X)
y_pred 

array(['구매', '비구매', '비구매', '구매', '비구매', '비구매', '구매', '비구매', '비구매', '구매',
       '구매', '비구매', '구매', '구매', '구매', '구매', '비구매', '비구매', '구매', '구매'],
      dtype=object)

In [19]:
# 실제 라벨값과 예측 라벨값을 이용해 평가 지표 계산
print(confusion_matrix(y, y_pred)) 
print(classification_report(y, y_pred))

[[8 0]
 [3 9]]
              precision    recall  f1-score   support

          구매       0.73      1.00      0.84         8
         비구매       1.00      0.75      0.86        12

    accuracy                           0.85        20
   macro avg       0.86      0.88      0.85        20
weighted avg       0.89      0.85      0.85        20



## 03)KNN regression

In [20]:
# 정수 인코딩
buy_int = np.array([0]*len(buy))
buy_int[buy.purchase=='구매']=1
buy_int[buy.purchase=='비구매']=0
buy.purchase = buy_int
buy.head()

,age,paycheck,purchase
0,26,160,1
1,35,210,0
2,26,220,0
3,29,260,1
4,22,110,0


In [21]:
buy_origin = buy
buy_origin.head()

,age,paycheck,purchase
0,26,160,1
1,35,210,0
2,26,220,0
3,29,260,1
4,22,110,0


In [22]:
# 정규화
scaler = StandardScaler()
buy = scaler.fit(buy).transform(buy)
buy = pd.DataFrame(buy)
buy.columns = ['age','income','purchase']
buy.head()

,age,income,purchase
0,-0.677439,-0.813024,1.224745
1,1.016158,-0.259946,-0.816497
2,-0.677439,-0.149331,-0.816497
3,-0.112906,0.293131,1.224745
4,-1.430149,-1.366101,-0.816497


In [25]:
# 피쳐 및 라벨 지정(피쳐=수입,구매여부 라벨=나이)
# 즉 수입과 구매여부에 따라 나이를 알아보겠다는 뜻
X = buy.iloc[:, [1,2]].values
y = buy.iloc[:, [0]].values

# train/test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16, 2), (4, 2), (16, 1), (4, 1))

In [26]:
# 모델 학습
model = KNeighborsRegressor() # 수치 예측
model.fit(X_train, y_train)

# 예측값 계산
y_pred = model.predict(X_test)

In [27]:
# r2 score 계산
model.score(X_test,y_test)

-0.4933333333333336

In [28]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)
# 애초에 데이터 자체가 모형에 맞지않으니...결과가 엉망
# r2 score가 음수라는건 단순히 평균값으로 예측한거보다 결과가 안좋다는 뜻

-0.4933333333333336